# The Battle of Neighborhoods at Thiruvananthapuram City

Import Neccessary Packages

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


Install geopy and folium and import the packages

In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [4]:
from geopy.geocoders import Nominatim
import folium

print('geopy and folium imported')

geopy and folium imported


Importing BeautifulSoup to get the data of Thiruvanathapuram City from the link https://www.indiatvnews.com/pincode/kerala/thiruvananthapuram/thiruvananthapuram-g.p.o.-

In [5]:
from bs4 import BeautifulSoup

Getting Thiruvanathapuram City Data (Office(Neighborhood), Taluk, State, Pincode) and copying it to Pandas DataFrame

In [6]:
urlt = "https://www.indiatvnews.com/pincode/kerala/thiruvananthapuram/thiruvananthapuram-g.p.o.-"
responset = requests.get(urlt)
print(responset.status_code)

200


In [7]:
st = BeautifulSoup(responset.text,'html.parser').find('table',{'class':"alt"})

In [8]:
df_t=pd.read_html(str(st))
df_t=pd.DataFrame(df_t[0])
df_t.head()

,Office,Taluk,District,State,Pincode
0,Adayamon,Chirayinkeezhu,Thiruvananthapuram,KERALA,695614
1,Alamcode,Chirayinkeezhu,Thiruvananthapuram,KERALA,695102
2,Aliyadu,Nedumangad,Thiruvananthapuram,KERALA,695607
3,Altharamoodu,Chirayinkeezhu,Thiruvananthapuram,KERALA,695102
4,Anakudy,Nedumangad,Thiruvananthapuram,KERALA,695606


Including the Neighborhoods only in Taluk - 'Thiruvananthapuram'

In [9]:
df_t1 = df_t[df_t['Taluk'] == 'Thiruvananthapuram']
df_t1.reset_index(drop=True,inplace=True)
df_t1.head()

,Office,Taluk,District,State,Pincode
0,Anayara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695029
1,Attakulangara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695023
2,Beemapally,Thiruvananthapuram,Thiruvananthapuram,KERALA,695008
3,Chakkai,Thiruvananthapuram,Thiruvananthapuram,KERALA,695024
4,Channamkara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301


Initial Descriptive statistics of Thiruvananthapuram City Data

In [10]:
df_t1.describe(include ='all')

,Office,Taluk,District,State,Pincode
count,115,115,115,115,115.000000
unique,115,1,1,1,NaN
top,Kuttichal,Thiruvananthapuram,Thiruvananthapuram,KERALA,NaN
freq,1,115,115,115,NaN
mean,NaN,NaN,NaN,NaN,695184.565217
std,NaN,NaN,NaN,NaN,241.041895
min,NaN,NaN,NaN,NaN,695001.000000
25%,NaN,NaN,NaN,NaN,695012.500000
50%,NaN,NaN,NaN,NaN,695027.000000
75%,NaN,NaN,NaN,NaN,695511.000000


Including Latitude and Longitude of each neighborhood using geopy

In [11]:
lat = []
lon = []
for area in df_t1['Pincode']:
    geolocator = Nominatim(user_agent="ny_explorer")
    try:
        location = geolocator.geocode(area)
        lat.append(location.latitude)
        lon.append(location.longitude)
    except:
        lat.append(0)
        lon.append(0)
print(len(lat), len(lon))

115 115


In [12]:
df_t1['Latitude'] = lat
df_t1['Longitude'] = lon
df_t1.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Office,Taluk,District,State,Pincode,Latitude,Longitude
0,Anayara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695029,8.511202,76.906659
1,Attakulangara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695023,8.479239,76.933606
2,Beemapally,Thiruvananthapuram,Thiruvananthapuram,KERALA,695008,8.494977,76.900764
3,Chakkai,Thiruvananthapuram,Thiruvananthapuram,KERALA,695024,8.487481,76.929942
4,Channamkara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376


In [13]:
df_t1.shape

(115, 7)

In [14]:
df_t1['Latitude'].replace(0, np.nan, inplace=True)
df_t1['Longitude'].replace(0,np.nan, inplace=True)
df_t1.dropna(subset=["Latitude"], axis=0, inplace=True)
df_t1.dropna(subset=["Longitude"], axis=0, inplace=True)
df_t1.reset_index(drop=True, inplace=True)
df_t1.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

,Office,Taluk,District,State,Pincode,Latitude,Longitude
0,Anayara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695029,8.511202,76.906659
1,Attakulangara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695023,8.479239,76.933606
2,Beemapally,Thiruvananthapuram,Thiruvananthapuram,KERALA,695008,8.494977,76.900764
3,Chakkai,Thiruvananthapuram,Thiruvananthapuram,KERALA,695024,8.487481,76.929942
4,Channamkara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376


In [15]:
df_t1.shape

(113, 7)

Renaming Office Column as Neighborhood

In [16]:
df_t1.rename(columns = {'Office':'Neighborhood'},inplace=True)
df_t1.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Neighborhood,Taluk,District,State,Pincode,Latitude,Longitude
0,Anayara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695029,8.511202,76.906659
1,Attakulangara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695023,8.479239,76.933606
2,Beemapally,Thiruvananthapuram,Thiruvananthapuram,KERALA,695008,8.494977,76.900764
3,Chakkai,Thiruvananthapuram,Thiruvananthapuram,KERALA,695024,8.487481,76.929942
4,Channamkara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376


Declaring Credentials for using FOURSQUARR 

In [17]:
CLIENT_ID = 'RK3OMTF5SFL2JC0DNHRLMSQZMMDNBCG1OKAPBWMC5ZES1XRD' # your Foursquare ID
CLIENT_SECRET = '2PROC11YU2C4P31D53JFOWFIVMVZXPMMYZOJLF0QUVI53FNF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RK3OMTF5SFL2JC0DNHRLMSQZMMDNBCG1OKAPBWMC5ZES1XRD
CLIENT_SECRET:2PROC11YU2C4P31D53JFOWFIVMVZXPMMYZOJLF0QUVI53FNF


Getting the venues of 1st neighborhood using Foursquare

In [18]:
lat1 = df_t1.loc[0, 'Latitude'] 
lon1 = df_t1.loc[0, 'Longitude']
print(lat1, lon1)

8.5112021 76.9066588


In [19]:
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat1,lon1,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RK3OMTF5SFL2JC0DNHRLMSQZMMDNBCG1OKAPBWMC5ZES1XRD&client_secret=2PROC11YU2C4P31D53JFOWFIVMVZXPMMYZOJLF0QUVI53FNF&v=20180605&ll=8.5112021,76.9066588&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f980c8474cc40265e290b60'},
 'response': {'headerLocation': 'Thiruvananthapuram',
  'headerFullLocation': 'Thiruvananthapuram',
  'headerLocationGranularity': 'city',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 8.515702104500004,
    'lng': 76.91120041957964},
   'sw': {'lat': 8.506702095499996, 'lng': 76.90211718042036}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e68be69b0fb8e94c7e9e3bc',
       'name': 'KMS Traders',
       'location': {'address': '15, World Market',
        'crossStreet': 'NH47 Bypass Rd',
        'lat': 8.511453447084737,
        'lng': 76.90517727823106,
        'labeledLatLngs': [{'label': 'display',
          'lat': 8.511453447084737,
          'lng': 76.90517727823106}],
        'distance':

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,KMS Traders,Farmers Market,8.511453,76.905177
1,Indus Motors Service Center (Venpalavattom),Auto Garage,8.508968,76.905312
2,Ojeen Caterers & Restaurants,Asian Restaurant,8.509049,76.904614
3,Ajwa Biriyani,Kerala Restaurant,8.508343,76.905489
4,K R Bakes,Bakery,8.508222,76.905520


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


Getting the nearby Venue details of all the neighborhoods using FOURSQUARE and then filtering out required columns

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
             
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
TVM_venues = getNearbyVenues(names=df_t1['Neighborhood'],
                                   latitudes=df_t1['Latitude'],
                                   longitudes=df_t1['Longitude']
                                  )

Getting the number of Venues extracted from FOURSQUARE for each neighborhood

In [26]:
TVM_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ambalathara,2,2,2,2,2,2
Anayara,8,8,8,8,8,8
Attakulangara,4,4,4,4,4,4
Attukkal,5,5,5,5,5,5
Beemapally,1,1,1,1,1,1
Chakkai,8,8,8,8,8,8
Channamkara,1,1,1,1,1,1
Chittattumukku,1,1,1,1,1,1
Cotton Hill,10,10,10,10,10,10


Creating a List to filter only those venues that belongs to restaurants

In [27]:
TVM_venues['Venue Category'].value_counts()

Indian Restaurant                27
Fast Food Restaurant             16
Bakery                           14
Hotel                            14
Vegetarian / Vegan Restaurant    10
Restaurant                        9
Beach                             9
Movie Theater                     8
Train Station                     8
Electronics Store                 7
Department Store                  7
Juice Bar                         7
Shopping Mall                     7
Café                              7
Diner                             6
Asian Restaurant                  6
Candy Store                       6
Auto Garage                       6
Motorcycle Shop                   6
Furniture / Home Store            6
Seafood Restaurant                5
Other Great Outdoors              5
Gym                               4
Clothing Store                    4
Park                              4
Supermarket                       4
Sporting Goods Shop               4
Stadium                     

In [28]:
Hot_List = ['Indian Restaurant',
            'Fast Food Restaurant',
            'Hotel'
            'Vegetarian / Vegan Restaurant',
            'Restaurant',
            'Asian Restaurant',
            'Diner',
            'Seafood Restaurant',
            'Chinese Restaurant',
            'Comfort Food Restaurant',
            'Burger Joint',
            'Food Court',
            'Middle Eastern Restaurant',
            'Food',
            'Breakfast Spot',
            'Thai Restaurant',
            'Pizza Place',
            'Halal Restaurant',
            'Kerala Restaurant',
            'Neighbourhood']

One hot encoding each Venue category

In [29]:
TVM_venues_onehot = pd.get_dummies(TVM_venues[['Venue Category']], prefix="", prefix_sep="")

TVM_venues_onehot['Neighbourhood'] = TVM_venues['Neighborhood'] 

fixed_columns = [TVM_venues_onehot.columns[-1]] + list(TVM_venues_onehot.columns[:-1])
TVM_venues_onehot = TVM_venues_onehot[fixed_columns]

TVM_venues_onehot.head()

,Neighbourhood,Accessories Store,Arcade,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,Badminton Court,Bagel Shop,Bakery,Beach,Boat or Ferry,Boutique,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Candy Store,Chinese Restaurant,City,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Department Store,Design Studio,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gym,Halal Restaurant,Herbs & Spices Store,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Juice Bar,Kerala Restaurant,Lake,Liquor Store,Market,Men's Store,Middle Eastern Restaurant,Motorcycle Shop,Movie Theater,Neighborhood,Other Great Outdoors,Park,Pizza Place,Planetarium,Playground,Record Shop,Resort,Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Soccer Field,Sporting Goods Shop,Stadium,Supermarket,Surf Spot,Tea Room,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant
0,Anayara,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Anayara,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Anayara,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Anayara,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Anayara,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Inluding the venues that only belongs to retaurants

In [30]:
TVM_venues_oneshot1 = TVM_venues_onehot.loc[:, TVM_venues_onehot.columns.isin(Hot_List)]
TVM_venues_oneshot1.head()

,Neighbourhood,Asian Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food,Food Court,Halal Restaurant,Indian Restaurant,Kerala Restaurant,Middle Eastern Restaurant,Pizza Place,Restaurant,Seafood Restaurant,Thai Restaurant
0,Anayara,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Anayara,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Anayara,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Anayara,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Anayara,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
TVM_venues_grouped = TVM_venues_oneshot1.groupby('Neighbourhood').mean().reset_index()
TVM_venues_grouped

,Neighbourhood,Asian Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food,Food Court,Halal Restaurant,Indian Restaurant,Kerala Restaurant,Middle Eastern Restaurant,Pizza Place,Restaurant,Seafood Restaurant,Thai Restaurant
0,Ambalathara,0.00,0.000000,0.000000,0.0,0.000000,0.500000,0.000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.0,0.500000,0.000,0.00
1,Anayara,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.125,0.00,0.0,0.000000,0.125,0.00
2,Attakulangara,0.00,0.000000,0.000000,0.0,0.250000,0.000000,0.000,0.0,0.0,0.000000,0.500000,0.000,0.00,0.0,0.000000,0.000,0.25
3,Attukkal,0.00,0.000000,0.000000,0.0,0.000000,0.200000,0.000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000,0.00
4,Beemapally,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.0,0.000000,1.000,0.00
5,Chakkai,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.125,0.0,0.0,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000,0.00
6,Channamkara,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000,0.00
7,Chittattumukku,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000,0.00
8,Cotton Hill,0.00,0.000000,0.000000,0.1,0.000000,0.000000,0.100,0.1,0.1,0.000000,0.100000,0.000,0.00,0.0,0.000000,0.000,0.00
9,Industrial Estate Thiruvananthapuram,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000,0.00


In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Getting the top 10 served restaurant types in each neighborhood

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

TVM_final = pd.DataFrame(columns=columns)
TVM_final['Neighbourhood'] = TVM_venues_grouped['Neighbourhood']

for ind in np.arange(TVM_venues_grouped.shape[0]):
    TVM_final.iloc[ind, 1:] = return_most_common_venues(TVM_venues_grouped.iloc[ind, :], num_top_venues)

TVM_final.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ambalathara,Restaurant,Diner,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Fast Food Restaurant,Food Court
1,Anayara,Asian Restaurant,Kerala Restaurant,Seafood Restaurant,Fast Food Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Thai Restaurant
2,Attakulangara,Indian Restaurant,Thai Restaurant,Comfort Food Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Diner,Fast Food Restaurant,Food Court
3,Attukkal,Diner,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Fast Food Restaurant,Food Court,Seafood Restaurant
4,Beemapally,Seafood Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court


# USING K MEANS CLUSTERING TO CLUSTER THE NEIGHBORHOODS BASED ON THE SIMILARITY OF RETAURANT TYPES BEING SERVED

In [34]:
kclusters = 8

TVM_final_clustering = TVM_venues_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TVM_final_clustering)

kmeans.labels_[0:10]

array([3, 0, 7, 0, 1, 6, 0, 0, 6, 0], dtype=int32)

In [35]:
TVM_final.insert(0, 'Cluster Labels', kmeans.labels_)

In [36]:
df_last = df_t1

df_last = df_last.join(TVM_final.set_index('Neighbourhood'), on='Neighborhood')

df_last

,Neighborhood,Taluk,District,State,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anayara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695029,8.511202,76.906659,0.0,Asian Restaurant,Kerala Restaurant,Seafood Restaurant,Fast Food Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Thai Restaurant
1,Attakulangara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695023,8.479239,76.933606,7.0,Indian Restaurant,Thai Restaurant,Comfort Food Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Diner,Fast Food Restaurant,Food Court
2,Beemapally,Thiruvananthapuram,Thiruvananthapuram,KERALA,695008,8.494977,76.900764,1.0,Seafood Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
3,Chakkai,Thiruvananthapuram,Thiruvananthapuram,KERALA,695024,8.487481,76.929942,6.0,Fast Food Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Food Court,Seafood Restaurant
4,Channamkara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376,0.0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
5,Chempazhanthy,Thiruvananthapuram,Thiruvananthapuram,KERALA,695587,8.576357,76.916342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cheruvikkal,Thiruvananthapuram,Thiruvananthapuram,KERALA,695011,8.530583,76.920586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Chittattumukku,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376,0.0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
8,Kaithamukku,Thiruvananthapuram,Thiruvananthapuram,KERALA,695024,8.487481,76.929942,6.0,Fast Food Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Food Court,Seafood Restaurant
9,Kaniyapuram,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376,0.0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant


In [37]:
df_last.dropna(subset=["Cluster Labels"], axis=0, inplace=True)
df_last.reset_index(drop=True, inplace=True)
df_last

,Neighborhood,Taluk,District,State,Pincode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anayara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695029,8.511202,76.906659,0.0,Asian Restaurant,Kerala Restaurant,Seafood Restaurant,Fast Food Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Thai Restaurant
1,Attakulangara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695023,8.479239,76.933606,7.0,Indian Restaurant,Thai Restaurant,Comfort Food Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Diner,Fast Food Restaurant,Food Court
2,Beemapally,Thiruvananthapuram,Thiruvananthapuram,KERALA,695008,8.494977,76.900764,1.0,Seafood Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
3,Chakkai,Thiruvananthapuram,Thiruvananthapuram,KERALA,695024,8.487481,76.929942,6.0,Fast Food Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Food Court,Seafood Restaurant
4,Channamkara,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376,0.0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
5,Chittattumukku,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376,0.0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
6,Kaithamukku,Thiruvananthapuram,Thiruvananthapuram,KERALA,695024,8.487481,76.929942,6.0,Fast Food Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Food Court,Seafood Restaurant
7,Kaniyapuram,Thiruvananthapuram,Thiruvananthapuram,KERALA,695301,8.583439,76.852376,0.0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
8,Karikkakom,Thiruvananthapuram,Thiruvananthapuram,KERALA,695021,8.501570,76.898918,0.0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
9,Karyavattom,Thiruvananthapuram,Thiruvananthapuram,KERALA,695581,8.563473,76.888868,0.0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant


In [38]:
df_last[["Cluster Labels"]] = df_last[["Cluster Labels"]].astype("int")
df_last.describe()

,Pincode,Latitude,Longitude,Cluster Labels
count,85.000000,85.000000,85.000000,85.000000
mean,695172.917647,8.519946,76.932474,1.729412
std,238.944963,0.061892,0.055634,2.485138
min,695001.000000,8.391405,76.683761,0.000000
25%,695014.000000,8.487481,76.898918,0.000000
50%,695026.000000,8.501715,76.943819,0.000000
75%,695317.000000,8.557140,76.962150,2.000000
max,695589.000000,8.774233,77.143603,7.000000


### CREATING A MAP FOLIUM WHICH DISTINGUSHES THE NEIGHBORHOOD BASED ON CLUSTER

In [39]:
map_clusters = folium.Map(location=[8.52, 76.93], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_last['Latitude'], df_last['Longitude'], df_last['Neighborhood'], df_last['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
df_last.loc[df_last['Cluster Labels'] == 0, df_last.columns[[1] + list(range(5, df_last.shape[1]))]]

,Taluk,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Thiruvananthapuram,8.511202,76.906659,0,Asian Restaurant,Kerala Restaurant,Seafood Restaurant,Fast Food Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Thai Restaurant
4,Thiruvananthapuram,8.583439,76.852376,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
5,Thiruvananthapuram,8.583439,76.852376,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
7,Thiruvananthapuram,8.583439,76.852376,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
8,Thiruvananthapuram,8.501570,76.898918,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
9,Thiruvananthapuram,8.563473,76.888868,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
10,Thiruvananthapuram,8.620820,76.893624,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
12,Thiruvananthapuram,8.620820,76.893624,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
13,Thiruvananthapuram,8.620820,76.893624,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant
14,Thiruvananthapuram,8.521660,76.938674,0,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court,Seafood Restaurant


In [41]:
df_last.loc[df_last['Cluster Labels'] == 1, df_last.columns[[1] + list(range(5, df_last.shape[1]))]]

,Taluk,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Thiruvananthapuram,8.494977,76.900764,1,Seafood Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
40,Thiruvananthapuram,8.494977,76.900764,1,Seafood Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
43,Thiruvananthapuram,8.494977,76.900764,1,Seafood Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
45,Thiruvananthapuram,8.494977,76.900764,1,Seafood Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court


In [42]:
df_last.loc[df_last['Cluster Labels'] == 2, df_last.columns[[1] + list(range(5, df_last.shape[1]))]]

,Taluk,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Thiruvananthapuram,8.565834,76.878349,2,Indian Restaurant,Restaurant,Halal Restaurant,Thai Restaurant,Fast Food Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner
18,Thiruvananthapuram,8.630357,76.898035,2,Indian Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
28,Thiruvananthapuram,8.630357,76.898035,2,Indian Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
29,Thiruvananthapuram,8.557140,76.922752,2,Indian Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
33,Thiruvananthapuram,8.482036,76.954829,2,Fast Food Restaurant,Restaurant,Indian Restaurant,Thai Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Food Court
38,Thiruvananthapuram,8.501715,76.951127,2,Indian Restaurant,Restaurant,Asian Restaurant,Breakfast Spot,Middle Eastern Restaurant,Fast Food Restaurant,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner
46,Thiruvananthapuram,8.497091,76.939363,2,Indian Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
47,Thiruvananthapuram,8.497091,76.939363,2,Indian Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
76,Thiruvananthapuram,8.489315,76.977156,2,Indian Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
77,Thiruvananthapuram,8.489315,76.977156,2,Indian Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court


In [43]:
df_last.loc[df_last['Cluster Labels'] == 3, df_last.columns[[1] + list(range(5, df_last.shape[1]))]]

,Taluk,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Thiruvananthapuram,8.43723,76.960415,3,Restaurant,Diner,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Fast Food Restaurant,Food Court
78,Thiruvananthapuram,8.43723,76.960415,3,Restaurant,Diner,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Fast Food Restaurant,Food Court


In [44]:
df_last.loc[df_last['Cluster Labels'] == 4, df_last.columns[[1] + list(range(5, df_last.shape[1]))]]

,Taluk,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Thiruvananthapuram,8.581167,76.871756,4,Indian Restaurant,Thai Restaurant,Food,Breakfast Spot,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Fast Food Restaurant,Food Court
